# National Parks Historical Weather
This project sought to identify typical monthly weather conditions for each of the United States' National Parks.

## Inspiration
While browsing the Google Cloud Platform's BigQuery datasets, I noted that they had a sample dataset of historical weather observations. Initially I though it would be interesting to combine that with their own natality sample dataset, but the metrics I was interested were too sparse.

Next I thought it might be interesting to track historical weather conditions along the Pacific Crest Trail, which runs from the border of Mexico, through California, Oregon, and Washinton to the border of Canada. The GIS data proved too difficult for the available timeframe. Looking at information for smaller trails, I finally decided that National Parks would be a good dataset to work with.

## Requirements
I already had a starting point in the form of the BigQuery dataset. Observations listed station IDs in a variety of formats, and initially I explored using the WMO IDs (WMO is World Meteorological Organization). However I quickly realized that many (but not all) of the IDs were corrupted, having an extra zero (0) at the end. So I decided to use the WBAN numbers, which were more common in the data, and are US-centric anyway (WBAN stands for Weather Bureau, Air Force, Navy).

Given this, I had to find a source of data that mapped those numbers to locations that could then be compared to the national park locations. A list of the national park latitude/longitude coordinates was quickly identified on latlong.net, but identifying the location of WBAN stations was much more difficult. Eventually I found a text file on an old FTP server that was still being updated with changes to the WBAN network. It was not pretty.

<hr>

# DATA SOURCES
## LATLONG.NET
### Extract
* A list of national parks with their decimal latitude and longitudes was found here:
* https://www.latlong.net/category/national-parks-236-42.html

The table found on their site was manually copy-pasted insto a spreadsheet application and saved out as a CSV file.

### Transform
The CSV file was loaded into a Pandas DataFrame. The Place Name column was replaced by a Name column, after splitting out the park's name from the full place name, which included state and country info that was not consistently formatted. Later, the closest WBAN station to each park that was also represented in the BigQuery data, would be added to another column of the DataFrame.

The dataframe was then saved out as the ```./Results/parks.json``` file, so that it could be loaded by script into the final database.

## NCDC / NCEI / NOAA
### Extract
* The Master Station History text file that tracks all changes to the WBAN network:
* https://www1.ncdc.noaa.gov/pub/data/inventories/MASTER-STN-HIST.TXT
* The 50 Mb file is also available in ZIP format at just under 4Mb:
* https://www1.ncdc.noaa.gov/pub/data/inventories/MASTER-STN-HIST.ZIP

The source file is raw text that creates space-delimited columns through space-padding. This file was downloaded and manually converted into a CSV file by taking advantage of the column-edit mode of some advanced text editors.

### Transform
While converting to CSV, numerous unused columns were removed from the data. A macro was then created to find the next entry with no WBAN number and remove that line, then allowed to execute repeatedly until the end of file was reached. Search and replace tools were then used to converted double-spaces into commas, then another pass converted single-spaces into commas. Finally the file could be saved out as a CSV file.

The CSV file was then loaded into a speadsheet application and sorted by WBAN number (asc) first, then by END_DATE (desc). Latitude and longitude values, which were in degree/minutes/seconds format spanning 6 columns, were converted to decimal representation in new columns. Finall, any gaps in a WBAN's historical record were calculated and recorded in a new column, while the largest such gap was tracked for later evaluation.  The CSV file was again saved out so that it could be loaded into a Pandas DataFrame.

Once in a DataFrame, I identified the number of unique WBAN numbers (which was 6871) to use as confirmation that the next transformation had gone correctly. First, .groupby() and .min() were used to identify the earlies START_DATE for each WBAN number. Duplicate rows, as determined by the WBAN column, were then dropped, keeping the last (i.e., newest) entry. Finally (for this phase), the START_DATES column was overwritten with the earliest dates that we had previously saved.

After a cursory evaluation of time gaps, I decided to throw out any stations with a time gap of more than a year. This was done using .drop() on the results of a complicated .loc() evaluation.

## GOOGLE CLOUD PLATFORM
### Extract
* The BigQuery GSOD Sample Dataset
* https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table
* General info on, using the public data, including limits (1TB free per month):
* https://cloud.google.com/bigquery/public-data/

The first step in using this dataset (beyond simply exploring how to use it efficiently), was to query a list of unique WBAN numbers that were represented in the sample data. Those results were copy-pasted into a spreadsheet application, then saved out as a CSV file to be read into a Pandas DataFrame. It was at this point that I realized that the WBAN numbers were also corrupt, but not as badly as the WMO identifiers. While engaged in the next step of transformation, any WBAN station numbers smaller than 100 were excluded from the list of options.

Later the BigQuery dataset would be queried for the average monthly value of mean temperature, mean visibility, mean wind speed, total precipitation, and snow depth, for the WBAN stations that are closest to each national park. The query was saved out as a JSON file, then loaded into a Pandas DataFrame.

### Transform
Lat/lng data was looked up from the master stations dataframe and applied to the list created from the first query. Then an efficient algorithm from StackOverflow was adapted to help find which of these WBAN stations were closest to each national park, and as noted earlier, appended to the park dataframe.

The station list with lat/lng data was save out as the ```./Results/stations.json``` file, so that ic could be loaded by script into the final database.

Imputation of NaN values from the second (primary) query was done to make them all 0, under the assumption that they were not applicable, and therefore simply not reported. Most (possibly all) such values were in the visibility and snow depth columns. Finally the wban_number column was renamed to just wban to match other dataframes.

The dataframe was then saved out as the ```./Results/weather.json``` file, so that it could be loaded by script into the final database.

<hr>

# LOAD
The ```npw_load.py``` script was written to create and load the final database with data from the saved out .json files in the ./Results directory.

This script uses SQLAlchemy's ORM to create 3 tables in a relational sqlite database,```./national_parks_weather.sqlite```. It is arranged like the following:

![ERD][ERD]

A sqlite database was chosen as a convenience for putting it up on Kaggle, allowing others to explore and use in their learnings.

[ERD]: ./erd.png